## 1.Import libraries

In [1]:
import pandas as pd
import numpy as np
import re

## 2. Load de data 

In [2]:
df = pd.read_csv("../input/attacks.csv",encoding = "ISO-8859-1")
df.head()

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
0,2018.06.25,25-Jun-2018,2018.0,Boating,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,...,White shark,"R. Collier, GSAF",2018.06.25-Wolfe.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.25,2018.06.25,6303.0,NaN,NaN
1,2018.06.18,18-Jun-2018,2018.0,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,Adyson McNeely,F,...,NaN,"K.McMurray, TrackingSharks.com",2018.06.18-McNeely.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.18,2018.06.18,6302.0,NaN,NaN
2,2018.06.09,09-Jun-2018,2018.0,Invalid,USA,Hawaii,"Habush, Oahu",Surfing,John Denges,M,...,NaN,"K.McMurray, TrackingSharks.com",2018.06.09-Denges.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.09,2018.06.09,6301.0,NaN,NaN
3,2018.06.08,08-Jun-2018,2018.0,Unprovoked,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,male,M,...,2 m shark,"B. Myatt, GSAF",2018.06.08-Arrawarra.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.08,2018.06.08,6300.0,NaN,NaN
4,2018.06.04,04-Jun-2018,2018.0,Provoked,MEXICO,Colima,La Ticla,Free diving,Gustavo Ramos,M,...,"Tiger shark, 3m",A .Kipper,2018.06.04-Ramos.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.04,2018.06.04,6299.0,NaN,NaN


In [3]:
df.dtypes

Case Number                object
Date                       object
Year                      float64
Type                       object
Country                    object
Area                       object
Location                   object
Activity                   object
Name                       object
Sex                        object
Age                        object
Injury                     object
Fatal (Y/N)                object
Time                       object
Species                    object
Investigator or Source     object
pdf                        object
href formula               object
href                       object
Case Number.1              object
Case Number.2              object
original order            float64
Unnamed: 22                object
Unnamed: 23                object
dtype: object

In [4]:
# How big is the dataset?
df.shape


(25723, 24)

**¿There are some nan instead of Na?** 

### 2.1. Loking for NaN 

In [5]:
#Percentage of NaN along rows
null_rows = df.isnull().sum(axis = 1).apply (lambda y: y/df.shape[1]).sort_values(ascending = False)
null_rows[:5]

# With this I have seen that there are many rows in  the data frame that do not have any data. In this way, I will eliminate 
    ## those rows that present a percentage of nulls = 1 to clean up the dataframe a bit.

12861    1.0
14368    1.0
14382    1.0
14381    1.0
14380    1.0
dtype: float64

In [6]:
# To remove this rows:
df_2 = df.dropna(axis = 0, how = "all")
df_2.shape

#The number of rows was reduced from 25723 to 8703

(8703, 24)

In [7]:
df_2.head()

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
0,2018.06.25,25-Jun-2018,2018.0,Boating,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,...,White shark,"R. Collier, GSAF",2018.06.25-Wolfe.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.25,2018.06.25,6303.0,NaN,NaN
1,2018.06.18,18-Jun-2018,2018.0,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,Adyson McNeely,F,...,NaN,"K.McMurray, TrackingSharks.com",2018.06.18-McNeely.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.18,2018.06.18,6302.0,NaN,NaN
2,2018.06.09,09-Jun-2018,2018.0,Invalid,USA,Hawaii,"Habush, Oahu",Surfing,John Denges,M,...,NaN,"K.McMurray, TrackingSharks.com",2018.06.09-Denges.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.09,2018.06.09,6301.0,NaN,NaN
3,2018.06.08,08-Jun-2018,2018.0,Unprovoked,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,male,M,...,2 m shark,"B. Myatt, GSAF",2018.06.08-Arrawarra.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.08,2018.06.08,6300.0,NaN,NaN
4,2018.06.04,04-Jun-2018,2018.0,Provoked,MEXICO,Colima,La Ticla,Free diving,Gustavo Ramos,M,...,"Tiger shark, 3m",A .Kipper,2018.06.04-Ramos.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.04,2018.06.04,6299.0,NaN,NaN


In [8]:
#Now I will check the percentage of NaN along columns
perc_null = df_2.isnull().sum().apply(lambda x: x/df_2.shape[0]).sort_values(ascending=False)

In [29]:
#Time to select a determined value to columns with high percentage of nulls and probably with posibilities to remove
perc_null[perc_null > 0.7]

#Given the high percentage of cells with NaN in these columns (Unnamed22n = 0.999885, Unnamed23 =  0.999770) i poceed to remove
    #them. In addition to these columns there are three more columns with bibliographic references that are not relevant for the 
    #task and i proceed to delet them too (pdf, href formula, href). 
    
df_3 = df_2.drop(['Unnamed: 22', 'Unnamed: 23', 'pdf', 'href formula', 'href', 'Name','Case Number.1','Injury', 'Case Number.2', 'Investigator or Source'], axis = 1) 


In [30]:
# Case Number and Date columns cointain the same information but with different structure. I proceed to remove one of them (Case
    #number column)

df_3 = df_3.drop(['Case Number'], axis = 1)
df_3.head()

,Date,Year,Type,Country,Area,Location,Activity,Sex,Age,Fatal (Y/N),Time,Species,original order
0,25-Jun-2018,2018.0,Boating,USA,California,"Oceanside, San Diego County",Paddling,F,57,N,18h00,White shark,6303.0
1,18-Jun-2018,2018.0,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,F,11,N,14h00 -15h00,NaN,6302.0
2,09-Jun-2018,2018.0,Invalid,USA,Hawaii,"Habush, Oahu",Surfing,M,48,N,07h45,NaN,6301.0
3,08-Jun-2018,2018.0,Unprovoked,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,M,NaN,N,NaN,2 m shark,6300.0
4,04-Jun-2018,2018.0,Provoked,MEXICO,Colima,La Ticla,Free diving,M,NaN,N,NaN,"Tiger shark, 3m",6299.0


In [31]:
# to avoid problems with spaces in columns names I remove and replace the spaces with underscore 
df_3.columns =df_3.columns.str.strip()
df_3.columns = df_3.columns.str.replace(' ', '_') 
df_3.columns


Index(['Date', 'Year', 'Type', 'Country', 'Area', 'Location', 'Activity',
       'Sex', 'Age', 'Fatal_(Y/N)', 'Time', 'Species', 'original_order'],
      dtype='object')

### 2.2. Now is time to clean the selected columns. 

To realice this work I will work column by column to make them as clean as possible.

###### Date column ====> 25-Jun-2018
Along this column i am interested only in the month, because of the year is represented in the next column and the day it is not important for my hypothesis

In [52]:
# con esto saco solo los meses de toda la información que tengo en esta columna

df_3['Date'] = df_3['Date'].astype(str)
df_3["Date"] = df_3[df_3.columns[0]].apply(lambda x: re.findall(r'\b[A-Z]\w{2}\b', x))  

#Para quitar los corchetes de los valores:
df_3['Date'] = df_3['Date'].str.get(0)
df_3


,Date,Year,Type,Country,Area,Location,Activity,Sex,Age,Fatal_(Y/N),Time,Species,original_order
0,Jun,2018.0,Boating,usa,California,"Oceanside, San Diego County",Paddling,F,57.00,N,night,White shark,6303.0
1,Jun,2018.0,Unprovoked,usa,Georgia,"St. Simon Island, Glynn County",Standing,F,11.00,N,afternoon,NaN,6302.0
2,Jun,2018.0,Invalid,usa,Hawaii,"Habush, Oahu",Surfing,M,48.00,N,day-morning,NaN,6301.0
3,Jun,2018.0,Unprovoked,australia,New South Wales,Arrawarra Headland,Surfing,M,27.32,N,NaN,2 m shark,6300.0
4,Jun,2018.0,Provoked,mexico,Colima,La Ticla,Free diving,M,27.32,N,NaN,"Tiger shark, 3m",6299.0
5,Jun,2018.0,Unprovoked,australia,New South Wales,"Flat Rock, Ballina",Kite surfing,M,27.32,N,NaN,NaN,6298.0
6,Jun,2018.0,Unprovoked,brazil,Pernambuco,"Piedade Beach, Recife",Swimming,M,18.00,Y,afternoon,Tiger shark,6297.0
7,May,2018.0,Unprovoked,usa,Florida,"Lighhouse Point Park, Ponce Inlet, Volusia County",Fishing,M,52.00,N,NaN,"Lemon shark, 3'",6296.0
8,May,2018.0,Unprovoked,usa,Florida,"Cocoa Beach, Brevard County",Walking,M,15.00,N,afternoon,"Bull shark, 6'",6295.0
9,May,2018.0,Unprovoked,usa,Florida,"Daytona Beach, Volusia County",Standing,M,12.00,N,afternoon,NaN,6294.0


###### Fatal column 

In [53]:
df_3["Fatal_(Y/N)"].unique()

array(['N', 'Y', nan, 'M', 'UNKNOWN', '2017', ' N', 'N ', 'y'],
      dtype=object)

In [54]:
# Fisrt of all put everything in capital letters and remove possible spaces

df_3["Fatal_(Y/N)"] = df_3["Fatal_(Y/N)"].str.upper()
df_3["Fatal_(Y/N)"] = df_3["Fatal_(Y/N)"].str.strip()
df_3["Fatal_(Y/N)"].value_counts()

N          4301
Y          1389
UNKNOWN      71
2017          1
M             1
Name: Fatal_(Y/N), dtype: int64

In [55]:
# Given the doubtfulness of the rest of the veracity of the rest of the data, I will classify them all as NaN
df_3["Fatal_(Y/N)"] = df_3["Fatal_(Y/N)"].replace(['2017', 'M', 'UNKNOWN' ], np.nan)
df_3["Fatal_(Y/N)"].unique()

array(['N', 'Y', nan], dtype=object)

###### Type column

In [56]:
df_3["Type"].unique()
df_3["Type"]  = df_3["Type"].replace('Boat\w*', "Boat", regex = True)
df_3["Type"].value_counts()

Unprovoked      4595
Provoked         574
Invalid          547
Boat             341
Sea Disaster     239
Questionable       2
Name: Type, dtype: int64

###### Activity column

In [57]:
df_3["Activity"].unique()

df_3["Activity"]= df_3["Activity"].str.lower()

df_3["Activity"].value_counts()
df_3["Activity"] = df_3["Activity"].replace([".*fish.*",".*Fish.*", "angling" ], "Fish", regex = True)
df_3["Activity"] = df_3["Activity"].replace([".*surf.*","paddleskiing" ], "Surf", regex = True)
df_3["Activity"] = df_3["Activity"].replace([".*swim.*", "bathing", "fell into the water", "treading water", ".*float.*"], "Swim", regex = True)
df_3["Activity"] = df_3["Activity"].replace([".*diving.*",  ".*under.*", ".*film.*", "snorkeling"], "Diving", regex = True)
df_3['Activity'] = df_3["Activity"].replace(['.*kayak.*','.*ship.*', '.*boat.*','.*board.*', 'canoeing','rowing','sailing','fishing for mackerel'],'On_boat',regex=True)
df_3["Activity"] = df_3["Activity"].replace([".*shore*.", "treading for clams", "standing", "walking", ".*play*.", "wading"], "Seashore_act.", regex = True)
df_3['Activity'] = df_3["Activity"].replace(['.{,30} shark.','shark related act.*'],'Shark_related_act.',regex=True)

rare_act = df_3["Activity"].value_counts().loc[df_3['Activity'].value_counts() < 5].index
df_3.loc[df_3['Activity'].isin(rare_act),'Activity'] = 'Rare_act.'

df_3["Activity"].value_counts()

Swim                       1349
Surf                       1233
Fish                       1226
Diving                      620
Rare_act.                   523
On_boat                     484
Seashore_act.               272
Shark_related_act.           19
sea disaster                 14
Seashore_act.ng               7
murder                        6
dFish feet in the water       5
Name: Activity, dtype: int64

###### Year column ====> 2018.0
The objective in this column is to keep only the year eliminating .0

In [58]:
print(type(df_3.loc[df_3.index[0], 'Year']))

<class 'numpy.float64'>


In [59]:
df_3['Year'] = df_3['Year'].replace('.0', ' ')
df_3.sample(10)

,Date,Year,Type,Country,Area,Location,Activity,Sex,Age,Fatal_(Y/N),Time,Species,original_order
7136,NaN,NaN,NaN,NaN,NaN,NaN,NaN,F,27.32,NaN,NaN,NaN,NaN
3826,Apr,1962.0,Unprovoked,mozambique,Gaza,Xai Xai,Swim,M,16.00,N,day-morning,NaN,2477.0
4523,NaN,1949.0,Unprovoked,new guinea,Calvados Archipelago,"Sabara Island, Calvados Chain 11.4S, 153E",NaN,M,27.32,Y,NaN,NaN,1780.0
5335,Jul,1914.0,Provoked,usa,Louisiana,New Orleans,Fish,M,27.32,N,NaN,NaN,968.0
2953,Jan,1983.0,Invalid,south africa,KwaZulu-Natal,Amanzimtoti,NaN,M,27.32,NaN,NaN,Shark involvement prior to death was not confi...,3350.0
3057,Aug,1980.0,Unprovoked,usa,North Carolina,"Ocean Isle, Brunswick County",Seashore_act.,F,10.00,N,NaN,NaN,3246.0
5772,Mar,1885.0,Unprovoked,south africa,KwaZulu-Natal,Kokstad District,Fish,M,27.32,N,NaN,3 m [10'] shark,531.0
813,Mar,2012.0,Unprovoked,usa,Florida,"Playalinda Beach, Brevard County",Surf,M,19.00,N,afternoon,4' shark,5490.0
6303,NaN,NaN,NaN,NaN,NaN,NaN,NaN,F,27.32,NaN,NaN,NaN,6305.0
4121,Jul,1959.0,Unprovoked,india,Orissa,Mouth of Devi River at Machgaon,NaN,F,27.32,Y,NaN,1.5 m to 1.8 m [5' to 6'] sharks,2182.0


###### Type column 
This column is OK

###### Country column
- There are countries and continents. To unify the contents I will work with continents

In [60]:
df_3['Country'].value_counts().head(20)

usa                 2229
australia           1338
south africa         579
papua new guinea     134
new zealand          128
brazil               112
bahamas              109
mexico                90
italy                 71
fiji                  65
philippines           62
reunion               60
new caledonia         53
cuba                  46
mozambique            45
spain                 44
egypt                 40
india                 40
croatia               34
japan                 34
Name: Country, dtype: int64

In [61]:
df_3['Country'] = df_3['Country'].str.strip()
df_3['Country'] = df_3['Country'].str.lower()
df_3['Country'].replace(to_replace=['.*OCEAN','.* SEA'],value=np.nan,regex=True,inplace=True)
df_3['Country'].replace(['BETWEEN PORTUGAL & INDIA','IRAN / IRAQ','EGYPT / ISRAEL','RED SEA?'],np.nan,inplace=True)
df_3['Country'].replace('EQUATORIAL GUINEA / CAMEROON','EQUATORIAL GUINEA',inplace=True)
df_3['Country'].replace('CEYLON (SRI LANKA)','SRI LANKA',inplace=True)
df_3['Country'].replace('UNITED ARAB EMIRATES (UAE)','UNITED ARAB EMIRATES',inplace=True)
df_3['Country'].value_counts().head(20)

usa                 2229
australia           1338
south africa         579
papua new guinea     134
new zealand          128
brazil               112
bahamas              109
mexico                90
italy                 71
fiji                  65
philippines           62
reunion               60
new caledonia         53
cuba                  46
mozambique            45
spain                 44
egypt                 40
india                 40
croatia               34
japan                 34
Name: Country, dtype: int64

###### Sex column 0.34 null values
I this case I try to reemplace the NaN data for a random selectinon of sex. We have different unique values: 

- 'F'
- 'M'
- **nan**
- **'M '**
- **'lli'**
- **'N'**
- **'.'**

marked in bld the characters that I proceed to remove

In [62]:
import random
fill_list = ["M", "F"]
df_3["Sex"] = df_3["Sex"].fillna(pd.Series(np.random.choice(fill_list)))


In [63]:
df_3.replace({"Sex": {"nan": " ", "lli": " ", "N": " ", ".": " " }})

,Date,Year,Type,Country,Area,Location,Activity,Sex,Age,Fatal_(Y/N),Time,Species,original_order
0,Jun,2018.0,Boat,usa,California,"Oceanside, San Diego County",Rare_act.,F,57.00,N,night,White shark,6303.0
1,Jun,2018.0,Unprovoked,usa,Georgia,"St. Simon Island, Glynn County",Seashore_act.,F,11.00,N,afternoon,NaN,6302.0
2,Jun,2018.0,Invalid,usa,Hawaii,"Habush, Oahu",Surf,M,48.00,N,day-morning,NaN,6301.0
3,Jun,2018.0,Unprovoked,australia,New South Wales,Arrawarra Headland,Surf,M,27.32,N,NaN,2 m shark,6300.0
4,Jun,2018.0,Provoked,mexico,Colima,La Ticla,Diving,M,27.32,N,NaN,"Tiger shark, 3m",6299.0
5,Jun,2018.0,Unprovoked,australia,New South Wales,"Flat Rock, Ballina",Surf,M,27.32,N,NaN,NaN,6298.0
6,Jun,2018.0,Unprovoked,brazil,Pernambuco,"Piedade Beach, Recife",Swim,M,18.00,Y,afternoon,Tiger shark,6297.0
7,May,2018.0,Unprovoked,usa,Florida,"Lighhouse Point Park, Ponce Inlet, Volusia County",Fish,M,52.00,N,NaN,"Lemon shark, 3'",6296.0
8,May,2018.0,Unprovoked,usa,Florida,"Cocoa Beach, Brevard County",Seashore_act.,M,15.00,N,afternoon,"Bull shark, 6'",6295.0
9,May,2018.0,Unprovoked,usa,Florida,"Daytona Beach, Volusia County",Seashore_act.,M,12.00,N,afternoon,NaN,6294.0


In [64]:
df_3["Sex"].unique()
df_3['Sex'] = df_3['Sex'].astype(str)

In [65]:
def replace_sex(col):
    chars =  'M ', 'lli', 'N', '.', 'nan'
    for c in chars:
        col = col.replace(c, random.choice(fill_list))
    return(col)
   

In [66]:
df_3["Sex"] = df_3["Sex"].apply(replace_sex)
df_3["Sex"].unique()

array(['F', 'M'], dtype=object)

###### Age
In this case I replace the NaN for the mean values of the mean. First of all  I try to clean all the data. We have several types of error in this column

- Ages in string format
- Ages separated by *&*, *or*, *to*, *>*
- Ages with *?*

In [67]:
df_3['Age'] = df_3['Age'].astype(str) # To work with re.findall function we need string not integer. 


pattern = "\d+"  #to remove those values with strings or simbols
df_3['Age'] = df_3['Age'].apply(lambda x: re.findall(pattern, x))
df_3

#but now we have some cells with multiple values. I asume that in this attack, one shark attacked many people, hence the  
    #different ages in the same cell
df_3.explode('Age')

df_3['Age'] = df_3['Age'].str.get(0)
df_3


,Date,Year,Type,Country,Area,Location,Activity,Sex,Age,Fatal_(Y/N),Time,Species,original_order
0,Jun,2018.0,Boat,usa,California,"Oceanside, San Diego County",Rare_act.,F,57,N,night,White shark,6303.0
1,Jun,2018.0,Unprovoked,usa,Georgia,"St. Simon Island, Glynn County",Seashore_act.,F,11,N,afternoon,NaN,6302.0
2,Jun,2018.0,Invalid,usa,Hawaii,"Habush, Oahu",Surf,M,48,N,day-morning,NaN,6301.0
3,Jun,2018.0,Unprovoked,australia,New South Wales,Arrawarra Headland,Surf,M,27,N,NaN,2 m shark,6300.0
4,Jun,2018.0,Provoked,mexico,Colima,La Ticla,Diving,M,27,N,NaN,"Tiger shark, 3m",6299.0
5,Jun,2018.0,Unprovoked,australia,New South Wales,"Flat Rock, Ballina",Surf,M,27,N,NaN,NaN,6298.0
6,Jun,2018.0,Unprovoked,brazil,Pernambuco,"Piedade Beach, Recife",Swim,M,18,Y,afternoon,Tiger shark,6297.0
7,May,2018.0,Unprovoked,usa,Florida,"Lighhouse Point Park, Ponce Inlet, Volusia County",Fish,M,52,N,NaN,"Lemon shark, 3'",6296.0
8,May,2018.0,Unprovoked,usa,Florida,"Cocoa Beach, Brevard County",Seashore_act.,M,15,N,afternoon,"Bull shark, 6'",6295.0
9,May,2018.0,Unprovoked,usa,Florida,"Daytona Beach, Volusia County",Seashore_act.,M,12,N,afternoon,NaN,6294.0


Once I have the column of clean years I process to replace the NaN. in this case, I will substitute the NaN for the mean of this column.

In [68]:
## first of all we need to convert str to int
df_3["Age"] = df_3["Age"].fillna(-1)
df_3["Age"] = df_3["Age"].astype(int)
df_3["Age"] = df_3["Age"].replace(-1, np.nan)

In [69]:
def mean_age(col):
    mean_age = col.mean()
    return mean_age

In [70]:
value_to_replace = mean_age(df_3['Age'])
value_to_replace = round(value_to_replace, 2)
value_to_replace

27.12

In [71]:
df_3["Age"] = df_3["Age"].fillna(value_to_replace)

###### Time

In [72]:
df_3['Time'] = df_3['Time'].str.strip()
df_3['Time'] = df_3['Time'].str.lower()
df_3.loc[df_3['Time'].notnull(),'Time'].sample(20)

607         afternoon
42          afternoon
114         afternoon
3181            night
3202      day-morning
4930            night
713         afternoon
3312        afternoon
539         afternoon
1267      day-morning
3816            night
2224        afternoon
1957        afternoon
5264    night-morning
2878      day-morning
1954      day-morning
1321        afternoon
3892      day-morning
286       day-morning
920         afternoon
Name: Time, dtype: object

In [73]:
df_3['Time'].replace(['.*evening.*','.*dusk.*',
                         '.*dark.*','.*night.*'],'night',regex=True,inplace=True)

df_3['Time'].replace(['pm','p\.m\.','.*midday.*','.*sunset.*','.*lunchtime.*',
                         '.*noon.*','.*daytime.*','.*afternoon.*'],'afternoon',regex=True,inplace=True)

df_3['Time'].replace(['a\.m\.''am','.*dawn.*',
                         '.*daybreak.*','.*morning.*'],'day-morning',regex=True,inplace=True)

In [74]:
#grouping 06:00 to 11:59 as day-morning because english is weird
df_3['Time'].replace(to_replace='.*0[6-9].?\d{2}.*',value='day-morning',regex=True,inplace=True)
df_3['Time'].replace(to_replace='.*1[0-1].?\d{2}.*',value='day-morning',regex=True,inplace=True)
#grouping 12:00 to 17:59 as afternoon
df_3['Time'].replace(to_replace='.*1[2-7].?\d{2}.*',value='afternoon',regex=True,inplace=True)
#grouping 18:00 to 23:59 as night
df_3['Time'].replace(to_replace='.*1[8-9].?\d{2}.*',value='night',regex=True,inplace=True)
df_3['Time'].replace(to_replace='.*2[0-3].?\d{2}.*',value='night',regex=True,inplace=True)
#grouping 00:00 to 05:59 as night-morning because english is weird
df_3['Time'].replace(to_replace='.*0[0-5].?\d{2}.*',value='night-morning',regex=True,inplace=True)

In [75]:
df_3[~df_3['Time'].isin(['day-morning','night-morning','night','afternoon'])]['Time'].count()

0

In [76]:
df_3.loc[~df_3['Time'].isin(['day-morning','night-morning','night','afternoon']),'Time'] = np.nan

In [77]:
df_3['Time'].value_counts(dropna=False)

NaN            5787
afternoon      1593
day-morning     923
night           400
Name: Time, dtype: int64

In [78]:
df_3

,Date,Year,Type,Country,Area,Location,Activity,Sex,Age,Fatal_(Y/N),Time,Species,original_order
0,Jun,2018.0,Boat,usa,California,"Oceanside, San Diego County",Rare_act.,F,57,N,night,White shark,6303.0
1,Jun,2018.0,Unprovoked,usa,Georgia,"St. Simon Island, Glynn County",Seashore_act.,F,11,N,afternoon,NaN,6302.0
2,Jun,2018.0,Invalid,usa,Hawaii,"Habush, Oahu",Surf,M,48,N,day-morning,NaN,6301.0
3,Jun,2018.0,Unprovoked,australia,New South Wales,Arrawarra Headland,Surf,M,27,N,NaN,2 m shark,6300.0
4,Jun,2018.0,Provoked,mexico,Colima,La Ticla,Diving,M,27,N,NaN,"Tiger shark, 3m",6299.0
5,Jun,2018.0,Unprovoked,australia,New South Wales,"Flat Rock, Ballina",Surf,M,27,N,NaN,NaN,6298.0
6,Jun,2018.0,Unprovoked,brazil,Pernambuco,"Piedade Beach, Recife",Swim,M,18,Y,afternoon,Tiger shark,6297.0
7,May,2018.0,Unprovoked,usa,Florida,"Lighhouse Point Park, Ponce Inlet, Volusia County",Fish,M,52,N,NaN,"Lemon shark, 3'",6296.0
8,May,2018.0,Unprovoked,usa,Florida,"Cocoa Beach, Brevard County",Seashore_act.,M,15,N,afternoon,"Bull shark, 6'",6295.0
9,May,2018.0,Unprovoked,usa,Florida,"Daytona Beach, Volusia County",Seashore_act.,M,12,N,afternoon,NaN,6294.0


###### Species

In [23]:
df_3['Species_'] = df_3['Species_'].str.lower()
df_3.loc[df_3['Species_'].notnull(),'Species_'].sample(40)

5074                                grey nurse shark, 4' 
2137                                        spinner shark
3829                                    1.8 m [6'] shark 
505                                         spinner shark
2831                                     1.5 m [5'] shark
967                       shark involvement not confirmed
1151                                       blue shark, 1m
4551                                         "grey shark"
847                                           white shark
2000                                  blacktip shark, 2' 
1139                       white shark, 12' to 15' female
2210                                          white shark
3273                                          white shark
4296                                             6' shark
835                                     white shark, 2.5m
626                                              3' shark
3601                                    4.6 m [15'] shark
1307          

The first step is replace the sharks of doubtful species by NaN

In [122]:
df_3['Species'].replace(['.*unconfirmed.*','.* not confirmed.*',
                                '.*questionable.*', '.*foot shark.*',
                                '.*small shark.*', '.*from shark.*',
                                '.*female shark.*', '.*large shark.*',
                                '.*but shark.*', '.*juvenile shark.*',
                                '.*the shark.*','.*two sharkK.*',
                                '.*another shark.*','.*KG SHARK.*',
                                '.*SEVERAL SHARK.*','.*same shark.*',
                                '.*ground shark.*', ' '],
                               np.nan,regex=True,inplace=True)

In [124]:
df_3.loc[df_3['Species'].notnull(),'Species'].sample(10)

1290               Small
1345              Medium
1911               Small
4412               Small
1219    Misidentificated
5354    Misidentificated
668               Medium
474                  Big
2752    Misidentificated
1703               Small
Name: Species, dtype: object

In [125]:
df_3['Species'] = df_3['Species'].replace(to_replace= ['.*[1-2].?\d.*', '.*Tiger.*', '.*grey.*', '.*gray.*', ".*[0-3].*"],value='Small',regex=True)

df_3['Species'] = df_3['Species'].replace(to_replace= ['.*White.*', '.*white.*', 'Blue',".*[6-9]'.*"],value='Big',regex=True)

df_3['Species'] = df_3['Species'].replace(to_replace = ['.*3.?\d.*', 'mako', ".*[4-5].*"], value = "Medium", regex = True)

df_3["Species"] = df_3["Species"].replace(to_replace = ['Invalid','Possibly', '.*Doubt.*', 'Unidentified ', '\w[or]'], value = "Misidentificated", regex = True)

df_3["Species"].value_counts()

Small               1402
Misidentificated    1092
Big                  540
Medium               422
Name: Species, dtype: int64

In [126]:
df_3

,Date,Year,Type,Country,Area,Location,Activity,Sex,Age,Fatal_(Y/N),Time,Species,original_order
0,Jun,2018.0,Boat,usa,California,"Oceanside, San Diego County",Rare_act.,F,57,N,night,Big,6303.0
1,Jun,2018.0,Unprovoked,usa,Georgia,"St. Simon Island, Glynn County",Seashore_act.,F,11,N,afternoon,NaN,6302.0
2,Jun,2018.0,Invalid,usa,Hawaii,"Habush, Oahu",Surf,M,48,N,day-morning,NaN,6301.0
3,Jun,2018.0,Unprovoked,australia,New South Wales,Arrawarra Headland,Surf,M,27,N,NaN,Small,6300.0
4,Jun,2018.0,Provoked,mexico,Colima,La Ticla,Diving,M,27,N,NaN,Small,6299.0
5,Jun,2018.0,Unprovoked,australia,New South Wales,"Flat Rock, Ballina",Surf,M,27,N,NaN,NaN,6298.0
6,Jun,2018.0,Unprovoked,brazil,Pernambuco,"Piedade Beach, Recife",Swim,M,18,Y,afternoon,Small,6297.0
7,May,2018.0,Unprovoked,usa,Florida,"Lighhouse Point Park, Ponce Inlet, Volusia County",Fish,M,52,N,NaN,Small,6296.0
8,May,2018.0,Unprovoked,usa,Florida,"Cocoa Beach, Brevard County",Seashore_act.,M,15,N,afternoon,Big,6295.0
9,May,2018.0,Unprovoked,usa,Florida,"Daytona Beach, Volusia County",Seashore_act.,M,12,N,afternoon,NaN,6294.0


In [ ]:
df_3.to_csv("data_visualization2.csv")